<a href="https://colab.research.google.com/github/sseunghyuns/deeplearning/blob/master/rps.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Computer Vision with CNNs

In [1]:
import urllib.request
import zipfile
import tensorflow as tf
from keras_preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint

In [2]:
url = 'https://storage.googleapis.com/download.tensorflow.org/data/rps.zip'
urllib.request.urlretrieve(url, 'rps.zip')
local_zip = 'rps.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('tmp/')
zip_ref.close()

In [3]:
TRAINING_DIR = "tmp/rps/"

## 데이터 처리
* 정규화 및 데이터 augmentation

In [4]:
training_datagen = ImageDataGenerator(
    rescale = 1.0 / 255.0,
    # rotation_range = 40,
    # width_shift_range = 0.2,
    # height_shift_range = 0.2,
    # shear_range = 0.2,
    # zoom_range = 0.2,
    # horizontal_flip = True,
    # fill_mode = 'nearest',
    validation_split = 0.2
)

In [5]:
# flow_from_directory
training_gen = training_datagen.flow_from_directory(
    TRAINING_DIR,
    batch_size = 128,
    target_size = (150,150),
    class_mode = 'categorical',
    subset = 'training'
)

validation_gen = training_datagen.flow_from_directory(
    TRAINING_DIR,
    batch_size = 128,
    target_size = (150,150),
    class_mode = 'categorical',
    subset = 'validation'
)

Found 2016 images belonging to 3 classes.
Found 504 images belonging to 3 classes.


## CNN 모델 생성

In [6]:
# 모델 생성
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)), # 148 x 148 x 32 x 3
    tf.keras.layers.MaxPooling2D(2, 2), # 74 x 74 x 32 x 3

    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'), # 72 x 72 x 32 x 3
    tf.keras.layers.MaxPooling2D(2, 2), # 36 x 36 x 32 x 3

    tf.keras.layers.Conv2D(16, (3, 3), activation='relu'), # 34 x 34 x 16 x 3
    tf.keras.layers.MaxPooling2D(2, 2), # 17 x 17 x 16 x 3

    tf.keras.layers.Conv2D(16, (3, 3), activation='relu'), #15 x 15 x 16 x 3
    tf.keras.layers.MaxPooling2D(2, 2), # 7  x 7 x 16 x 3

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(128, activation='relu'),
    # YOUR CODE HERE, BUT END WITH A 3 Neuron Dense, activated by softmax
    tf.keras.layers.Dense(3, activation='softmax') # 3개의 label로 나뉘어져야 한다. 
])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 72, 72, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 34, 34, 16)        4624      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 17, 17, 16)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 15, 15, 16)        2

In [7]:
# 모델 compile
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['acc'])

# 모델 checkpoint
check_path = 'ck.ckpt'
checkpoint = ModelCheckpoint(
    filepath = check_path,
    save_best_only = True,
    save_weights_only = True,
    monitor = 'val_loss',
    verbose =1
)

In [8]:
# 모델 학습
model.fit(training_gen,
          validation_data = (validation_gen),
          epochs = 30,
          steps_per_epoch = len(training_gen),
          validation_steps = len(validation_gen),
          callbacks = [checkpoint])


model.load_weights(check_path)

Epoch 1/30
16/16 [==============================] - ETA: 0s - loss: 1.0152 - acc: 0.4965
Epoch 00001: val_loss improved from inf to 0.74407, saving model to ck.ckpt
16/16 [==============================] - 7s 458ms/step - loss: 1.0152 - acc: 0.4965 - val_loss: 0.7441 - val_acc: 0.7063
Epoch 2/30
16/16 [==============================] - ETA: 0s - loss: 0.5051 - acc: 0.8199
Epoch 00002: val_loss improved from 0.74407 to 0.42504, saving model to ck.ckpt
16/16 [==============================] - 7s 452ms/step - loss: 0.5051 - acc: 0.8199 - val_loss: 0.4250 - val_acc: 0.7679
Epoch 3/30
16/16 [==============================] - ETA: 0s - loss: 0.1607 - acc: 0.9474
Epoch 00003: val_loss improved from 0.42504 to 0.28899, saving model to ck.ckpt
16/16 [==============================] - 7s 448ms/step - loss: 0.1607 - acc: 0.9474 - val_loss: 0.2890 - val_acc: 0.8452
Epoch 4/30
16/16 [==============================] - ETA: 0s - loss: 0.0564 - acc: 0.9821
Epoch 00004: val_loss did not improve from 0.